In [ ]:
# instructions:
# download pix3d
# fix models in pix3d which have multiple obj files (no single "model.obj"); rename one of them to be "model.obj"
# For chair class:
# IKEA_JULES_1/model_-0.248241748473_0.131155721985.obj
# IKEA_MARKUS/model_-0.87337964205_-0.0723611270545.obj
# IKEA_PATRIK/model_-0.104274359838_-0.0651591010433.obj
# IKEA_SKRUVSTA/model_0.398123524547_0.0140008021241.obj
# IKEA_SNILLE_1/model_0.622225291388_0.0492453107028.obj

# get branch of occnet for processing
# follow instructions in external (download opengl, glew, etc and build) and set pythonpath
# run bash dataset_pix3d/build.sh
# place pix3d.build into data/external (only 4_pointcloud and 4_points are necessary)

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import json
import pprint
import subprocess
from PIL import Image
import PIL.ImageOps    


In [13]:
PIX3D_DIR = "../data/external/pix3d"
PIX3D_POINTS_DIR = "../data/external/pix3d.build"

OUT_DIR = "../data/pix3d_processed"

classes = ['chair']

In [3]:
with open(os.path.join(PIX3D_DIR, "pix3d.json")) as f:
    pix3d_data_json = json.loads(f.read())
# convert list of dicts into a dict (keyed by image path) of dicts
pix3d_data_dict = { entry["img"]:entry for entry in pix3d_data_json}

In [ ]:
# opens image, and processes it by 
    

In [14]:
# copies files from pix3d and pix3d.build into new pix3d_processed folder.
# images are cropped to have the chair in the center; versions with and without BG are provided;
# points are rotated 90 degrees CCW about y-axis to orient chairs to shapenet (need to check if this is the case for other classes as well)
# https://stackoverflow.com/questions/11142851/adding-borders-to-an-image-using-python
for class_name in classes:
    for root, dirs, files in os.walk(os.path.join(PIX3D_DIR, "img", class_name)):
        for f in files:
            
            # getting paths of assets
            image_name = f.replace(".jpg","").replace(".jpeg", "").replace(".png","")
            image_path = os.path.join(root, f)
            image_dict = pix3d_data_dict[image_path.replace(PIX3D_DIR+'/', "")]
            mask_path = os.path.join(PIX3D_DIR, image_dict['mask'])
            model_name = image_dict['model'].split('/')[2]
            pointcloud_path = os.path.join(PIX3D_POINTS_DIR, class_name, "4_pointcloud", "{}.npz".format(model_name))
            points_path = os.path.join(PIX3D_POINTS_DIR, class_name, "4_points", "{}.npz".format(model_name))
            
            processed_dir = os.path.join(OUT_DIR, class_name, image_name)
            processed_img_dir = os.path.join(processed_dir, "img_pix3d")
            
            if not os.path.exists(processed_img_dir):
                os.makedirs(processed_img_dir)
                
            # ------------------------------------------------------------------------------
            # creating image
            img = Image.open(image_path)
            img_width = image_dict['img_size'][0]
            img_height = image_dict['img_size'][1]
            
            # create expanded version of img
            new_size = (max(img_height, img_width)+100, max(img_height, img_width)+100)
            img_bordered = Image.new("RGB", new_size, (255, 255, 255))
            img_bordered.paste(img, (int((new_size[0]-image_dict['img_size'][0])/2),int((new_size[1]-image_dict['img_size'][1])/2)))
            
            # processing mask
            mask = Image.open(mask_path)
            img_masked = Image.composite(Image.new("RGB", image_dict['img_size'], (255, 255, 255)), img, PIL.ImageOps.invert(mask))
            img_bordered_mask = Image.new("RGB", new_size, (255, 255, 255))
            img_bordered_mask.paste(img_masked, (int((new_size[0]-image_dict['img_size'][0])/2),int((new_size[1]-image_dict['img_size'][1])/2)))
            
            # creating square, centered crop
            bbox_height = image_dict['bbox'][3] - image_dict['bbox'][1]
            bbox_width = image_dict['bbox'][2] - image_dict['bbox'][0]
            
            # adjusting bbox to match after expanding image
            if img_height > img_width:
                img_new_bbox = [image_dict['bbox'][0]+50+(img_height-img_width)/2, image_dict['bbox'][1]+50, image_dict['bbox'][2]+50+(img_height-img_width)/2, image_dict['bbox'][3]+50]
            else:
                img_new_bbox = [image_dict['bbox'][0]+50, image_dict['bbox'][1]+50+(img_width-img_height)/2, image_dict['bbox'][2]+50, image_dict['bbox'][3]+50+(img_width-img_height)/2]
            
            # adjusting bbox so it's a square
            #TODO: correct possible off-by-one error in image size
            if bbox_height > bbox_width:
                img_new_bbox[0]-=(bbox_height-bbox_width)/2
                img_new_bbox[2]+=(bbox_height-bbox_width)/2
            else:
                img_new_bbox[1]-=(bbox_width-bbox_height)/2
                img_new_bbox[3]+=(bbox_width-bbox_height)/2
            
            img_bordered = img_bordered.crop(img_new_bbox)
            img_bordered.save(os.path.join(processed_img_dir,'bg.jpg'))
            img_bordered_mask = img_bordered_mask.crop(img_new_bbox)
            img_bordered_mask.save(os.path.join(processed_img_dir,'no_bg.jpg'))
            

            # ------------------------------------------------------------------------------
            # creating points/pointclouds
            # y-axis ccw by 90 deg
            rot_mat = np.array([[0,0,1],[0,1,0],[-1,0,0]])
            pc = np.load(pointcloud_path)
            np.savez(os.path.join(processed_dir,'pointcloud.npz'), points=pc["points"] @ rot_mat,
                     normals=pc["normals"] @ rot_mat, loc=pc["loc"], scale=pc["scale"])
            
            p = np.load(points_path)
            np.savez(os.path.join(processed_dir,'points.npz'), points=p["points"] @ rot_mat,
                     occupancies=p["occupancies"], loc=p["loc"], scale=p["scale"])  
            
            
            

In [22]:
# create split lsts. For pix3d, it's only a test set
for class_name in classes:
    class_img_codes = [ code.split('/')[2][:4] for code in list(pix3d_data_dict.keys()) if class_name in code ]
    with open (os.path.join(OUT_DIR, class_name, "train.lst"), 'w') as f:
        pass
    with open (os.path.join(OUT_DIR, class_name, "val.lst"), 'w') as f:
        pass
    with open (os.path.join(OUT_DIR, class_name, "test.lst"), 'w') as f:
        for img_code in class_img_codes:
            f.write("{}\n".format(img_code))